## Initial Model ##

In [21]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

In [22]:
df = pd.read_csv("arithmetic_dataset.csv")
train_df, val_df = train_test_split(df, test_size=0.1)

In [23]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

In [24]:
# 5️⃣ Tokenize datasets
max_len = 32

# Add the "solve: " prefix to help the T5 model understand the task
train_inputs = ["solve: " + p for p in train_df['problem']]
val_inputs = ["solve: " + p for p in val_df['problem']]

# Tokenize input problems
train_encodings = tokenizer(
    train_inputs,
    padding='max_length',
    truncation=True,
    max_length=max_len,
    return_tensors="pt"
)

val_encodings = tokenizer(
    val_inputs,
    padding='max_length',
    truncation=True,
    max_length=max_len,
    return_tensors="pt"
)


In [25]:
# Tokenize target solutions
train_labels = tokenizer(
    train_df['solution'].astype(str).tolist(),
    padding='max_length',
    truncation=True,
    max_length=max_len,
    return_tensors="pt"
).input_ids

val_labels = tokenizer(
    val_df['solution'].astype(str).tolist(),
    padding='max_length',
    truncation=True,
    max_length=max_len,
    return_tensors="pt"
).input_ids


In [26]:
# 6️⃣ Create PyTorch Dataset
class ArithmeticDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

# Initialize datasets
train_dataset = ArithmeticDataset(train_encodings, train_labels)
val_dataset = ArithmeticDataset(val_encodings, val_labels)


In [34]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",   # ✅ works in v4.55.4
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5
)

print("✅ TrainingArguments initialized successfully!")


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

In [35]:
import torch
import transformers
import accelerate

print("PyTorch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)


PyTorch: 2.7.1+cpu
Transformers: 4.55.4
Accelerate: 1.10.0
